<a href="https://colab.research.google.com/github/ekerintaiwoa/MediaApp/blob/master/ofline_aiagent2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: create an ai agents that allows users to upload a book, ask question about the book,Summarize Chapters or Sections,Study and Review

# This code requires several libraries that might not be installed by default
# on a standard Colab instance. We'll install them first.

# Install necessary libraries
!pip install PyMuPDF  # For handling PDF files
!pip install nltk  # For natural language processing tasks like tokenization
!pip install scikit-learn  # For text vectorization (TF-IDF)
!pip install faiss-cpu  # For efficient similarity search (vector database)
!pip install transformers  # For using powerful language models
!pip install torch  # PyTorch is a dependency for transformers

import fitz  # PyMuPDF
import nltk
import re
import faiss
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None
        self.tfidf_matrix = None
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None

        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None


    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            return

        section_texts = list(self.chapter_sections.values())
        print(f"Vectorizing {len(section_texts)} sections...")

        # Using TF-IDF for simplicity; consider Sentence Transformers for better semantic search
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
        self.tfidf_matrix = self.vectorizer.fit_transform(section_texts)

        # Create FAISS index
        dimension = self.tfidf_matrix.shape[1]
        self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
        self.index.add(self.tfidf_matrix.astype('float32')) # FAISS requires float32
        print("FAISS index created.")


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."

        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        question_vec = self.vectorizer.transform([question]).toarray().astype('float32')
        D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section

        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        section_titles = list(self.chapter_sections.keys())
        relevant_section_title = section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title]

        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context)
            return f"Answer: {answer['answer']} (Source: {relevant_section_title}, Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section:\n{relevant_section_text[:500]}..." # Show beginning of section

    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

# --- How to use the agent in Colab ---

# 1. Mount Google Drive to access your files (optional, but common)
from google.colab import drive
drive.mount('/content/drive')

# 2. Create an instance of the BookQA agent
agent = BookQA()

# 3. Upload a book (replace '/content/drive/My Drive/your_book.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/My Drive/your_book.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# --- Now you can interact with the agent ---

# Example 1: Ask a question
if agent.book_text:
    question1 = "What is the main topic of the introduction?"
    answer1 = agent.ask_question(question1)
    print(f"\nQuestion: {question1}")
    print(answer1)

    question2 = "What is the author's perspective on [some topic mentioned in your book]?"
    answer2 = agent.ask_question(question2)
    print(f"\nQuestion: {question2}")
    print(answer2)

    # Example 2: List sections
    print("\nListing sections:")
    agent.list_sections()

    # Example 3: Summarize a section (replace '1' with the actual section number or part of the title)
    section_to_summarize = "1" # <--- **CHANGE THIS TO A SECTION NUMBER OR TITLE FROM `list_sections()`**
    print(f"\nAttempting to summarize section: {section_to_summarize}")
    summary = agent.summarize_section(section_to_summarize)
    print(summary)

    # Example 4: Enter Study/Review mode
    print("\nEntering Study/Review mode:")
    # agent.study_review() # Uncomment this line to start the interactive study mode

else:
    print("\nBook was not loaded successfully. Please check the file path and format.")



In [1]:

# Install necessary libraries
!pip install PyMuPDF  # For handling PDF files
!pip install nltk  # For natural language processing tasks like tokenization
!pip install scikit-learn  # For text vectorization (TF-IDF)
!pip install faiss-cpu  # For efficient similarity search (vector database)
!pip install transformers  # For using powerful language models
!pip install torch  # PyTorch is a dependency for transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [2]:
import fitz  # PyMuPDF
import nltk
import re
import faiss
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [35]:
# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

import fitz  # PyMuPDF
import nltk
import re
import faiss
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None
        self.tfidf_matrix = None
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None

        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.vectorizer = None
            self.tfidf_matrix = None
            self.index = None
            return

        # Ensure all values in chapter_sections are strings, clean them, and filter empty ones
        valid_section_texts = []
        valid_section_titles = []
        print("Cleaning and filtering sections for vectorization:")
        for title, text in self.chapter_sections.items():
            if isinstance(text, str) and text.strip():
                cleaned_text = self.clean_text(text) # Clean the text
                if cleaned_text: # Only add if cleaned text is not empty
                    valid_section_texts.append(cleaned_text)
                    valid_section_titles.append(title) # Keep track of titles for valid sections
                else:
                    print(f"Warning: Section '{title}' became empty after cleaning. Skipping for vectorization.")
            else:
                print(f"Warning: Section '{title}' is not a valid string or is empty. Skipping for vectorization.")

        print(f"Found {len(valid_section_texts)} valid sections after cleaning and filtering.")

        if not valid_section_texts:
            print("No valid sections to vectorize after filtering.")
            self.vectorizer = None
            self.tfidf_matrix = None
            self.index = None
            return

        print(f"Vectorizing {len(valid_section_texts)} valid sections...")

        # Using TF-IDF for simplicity; consider Sentence Transformers for better semantic search
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
        try:
            self.tfidf_matrix = self.vectorizer.fit_transform(valid_section_texts)

            # Create FAISS index
            dimension = self.tfidf_matrix.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(self.tfidf_matrix.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = valid_section_titles
            else:
                print("TF-IDF matrix has no features after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during TF-IDF vectorization: {e}")
            self.vectorizer = None
            self.tfidf_matrix = None
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.vectorizer or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Clean the question before vectorizing for search
            cleaned_question = self.clean_text(question)
            if not cleaned_question:
                 return "Please provide a valid question after cleaning."

            question_vec = self.vectorizer.transform([cleaned_question]).toarray().astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error transforming question for search: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

In [29]:

# --- How to use the agent in Colab ---

# 1. Mount Google Drive to access your files (optional, but common)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# 2. Create an instance of the BookQA agent
agent = BookQA()


Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.
Models initialized.


In [31]:

# 3. Upload a book (replace '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Vectorizing 1 valid sections...
Error during TF-IDF vectorization: setting an array element with a sequence.


In [14]:
# --- Now you can interact with the agent ---

# Example 1: Ask a question
if agent.book_text:
    question1 = "BRIAN TRACY IDEAS TO LIVE BY"
    answer1 = agent.ask_question(question1)
    print(f"\nQuestion: {question1}")
    print(answer1)


In [18]:
# 2. Create an instance of the BookQA agent
agent = BookQA()

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.
Models initialized.


In [20]:
# 3. Upload a book (replace '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Vectorizing 1 sections...
Error reading file: setting an array element with a sequence.


In [26]:
# 3. Upload a book (replace '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/627260606-chatbotdoc (1).pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/627260606-chatbotdoc (1).pdf
Reading 97 pages from /content/drive/MyDrive/627260606-chatbotdoc (1).pdf...
Finished reading book.
Processing book text...
Identified 2 chapters/sections or chunks.
Vectorizing 2 sections...
Error reading file: setting an array element with a sequence.


In [25]:
# --- Now you can interact with the agent ---

# Example 1: Ask a question
if agent.book_text:
    question1 = "What is the main topic of the introduction?"
    answer1 = agent.ask_question(question1)
    print(f"\nQuestion: {question1}")
    print(answer1)

    question2 = "What are the key steps for setting goals?" # Example question related to the book title
    answer2 = agent.ask_question(question2)
    print(f"\nQuestion: {question2}")
    print(answer2)

    # Example 2: List sections
    print("\nListing sections:")
    agent.list_sections()

    # Example 3: Summarize a section (replace '1' with the actual section number or part of the title)
    # Check the output of list_sections() to find a section to summarize.
    # For now, let's try summarizing the first identified section (assuming there's at least one)
    if agent.chapter_sections:
        first_section_key = list(agent.chapter_sections.keys())[0]
        print(f"\nAttempting to summarize section: {first_section_key}")
        summary = agent.summarize_section(first_section_key)
        print(summary)
    else:
        print("\nNo sections available to summarize.")


    # Example 4: Enter Study/Review mode
    print("\nEntering Study/Review mode:")
    # agent.study_review() # Uncomment this line to start the interactive study mode

else:
    print("\nBook was not loaded successfully. Please check the file path and format.")


Book was not loaded successfully. Please check the file path and format.


In [28]:
# 3. Upload a book (replace '/content/drive/MyDrive/627260606-chatbotdoc (1).pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/627260606-chatbotdoc (1).pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/627260606-chatbotdoc (1).pdf
Reading 97 pages from /content/drive/MyDrive/627260606-chatbotdoc (1).pdf...
Finished reading book.
Processing book text...
Identified 2 chapters/sections or chunks.
Vectorizing 2 sections...
Error reading file: setting an array element with a sequence.


In [33]:
# 3. Upload a book (replace '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Vectorizing 1 valid sections...
Error during TF-IDF vectorization: setting an array element with a sequence.


In [36]:
# 3. Upload a book (replace '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)


Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Vectorizing 1 valid sections...
Error during TF-IDF vectorization: setting an array element with a sequence.


# Task
Explain the error "setting an array element with a sequence" encountered during TF-IDF vectorization when processing PDF files "/content/drive/MyDrive/627260606-chatbotdoc (1).pdf" and "/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf". If possible, fix the error in the provided code and incorporate the changes. Otherwise, diagnose the error.

## Install sentence transformers

### Subtask:
Add the necessary library for Sentence Transformers.


**Reasoning**:
The subtask is to add the necessary library for Sentence Transformers. This requires installing the `sentence-transformers` package using pip. This needs to be in a separate code cell as per the instructions.



In [37]:
!pip install sentence-transformers

## Update `initialize models`

### Subtask:
Load a Sentence Transformer model instead of relying solely on TF-IDF.


**Reasoning**:
Import the necessary class for Sentence Transformers and initialize the Sentence Transformer model in the `initialize_models` method, replacing the TF-IDF vectorizer initialization.



In [38]:
from sentence_transformers import SentenceTransformer

class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None # This will be replaced by the embedding model
        self.tfidf_matrix = None # This will be replaced by the embeddings
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None
        self.embedding_model = None # New attribute for Sentence Transformer model


        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

        # Initialize Sentence Transformer model for embeddings
        try:
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence Transformer model loaded.")
        except Exception as e:
            print(f"Error loading Sentence Transformer model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.embedding_model = None


    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index using Sentence Transformers."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return
        if not self.embedding_model:
            print("Sentence Transformer model not loaded. Cannot vectorize sections.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return


        section_texts = list(self.chapter_sections.values())
        print(f"Creating embeddings for {len(section_texts)} sections...")

        try:
            # Create embeddings using the Sentence Transformer model
            embeddings = self.embedding_model.encode(section_texts, convert_to_numpy=True)
            self.tfidf_matrix = embeddings # Store embeddings (renamed from tfidf_matrix)

            # Create FAISS index
            dimension = embeddings.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(embeddings.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = list(self.chapter_sections.keys()) # Store original titles
            else:
                print("Embeddings have no dimensions after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during embedding creation or FAISS index creation: {e}")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.embedding_model or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Create embedding for the question
            question_vec = self.embedding_model.encode([question], convert_to_numpy=True).astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error creating question embedding or searching index: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

## Review and refine

### Subtask:
Test the updated agent with the problematic PDF and potentially other documents. Adjust text splitting, model choice, or search parameters as needed.


**Reasoning**:
Re-instantiate the BookQA agent to load the updated code and then attempt to upload one of the problematic PDF files to test if the error is resolved.



In [39]:
# 2. Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# 3. Upload a book (replace the path with one of the problematic PDF files)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # Using one of the problematic files
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# If upload is successful, list sections to confirm processing
if agent.chapter_sections:
    print("\nBook processed successfully. Listing sections:")
    agent.list_sections()
else:
    print("\nBook processing failed.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence Transformer model loaded.
Models initialized.

Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Creating embeddings for 1 sections...
FAISS index created.

Book processed successfully. Listing sections:
Chapters/Sections:
1. Introduction/Beginning


**Reasoning**:
The previous step successfully loaded the book and created embeddings. Now, test the `ask_question`, `list_sections`, and `summarize_section` methods to ensure the agent is functional with the loaded book.



In [40]:
# Test the ask_question method
question1 = "What are the main ideas presented in the book?"
answer1 = agent.ask_question(question1)
print(f"\nQuestion: {question1}")
print(answer1)

question2 = "How to set goals?"
answer2 = agent.ask_question(question2)
print(f"\nQuestion: {question2}")
print(answer2)

# Test the list_sections method (already done in the previous step, but can be repeated)
print("\nListing sections again:")
agent.list_sections()

# Test the summarize_section method
# Assuming there's at least one section, summarize the first one.
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to summarize section: {first_section_key}")
    summary = agent.summarize_section(first_section_key)
    print(summary)
else:
    print("\nNo sections available to summarize.")


Searching for answer to: 'What are the main ideas presented in the book?'
Most relevant section: 'Introduction/Beginning'

Question: What are the main ideas presented in the book?
Answer: The more reasons you have for 
achieving your goal (Source: 'Introduction/Beginning', Score: 0.03)
Searching for answer to: 'How to set goals?'
Most relevant section: 'Introduction/Beginning'

Question: How to set goals?
Answer: act as if it were impossible to fail (Source: 'Introduction/Beginning', Score: 0.01)

Listing sections again:
Chapters/Sections:
1. Introduction/Beginning

Attempting to summarize section: Introduction/Beginning
Summarizing section: 'Introduction/Beginning'
Splitting section into 68 chunks for summarization...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarized chunk 1/68


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarized chunk 2/68


...

Summarized chunk 65/68


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarized chunk 66/68


Your max_length is set to 150, but your input_length is only 131. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarized chunk 67/68
Summarized chunk 68/68
Summary of 'Introduction/Beginning':
BRIAN TRACY GOAL PLANNER M A X I M U M A C H I E V E M E N T G O A L P L A N N N E R . the great secret of success is that there are no secrets of success .
there are no limits on what you can achieve with your life, except the limits you accept in your own mind . you are a potential genius; there is no problem you cannot solve, and no answer you cannot fi nd some- where .
Brian Tracy has consulted for more than 1,000 companies and addressed more than 3,000,000 people in 4,000 talks and seminars . he has studied, researched, written and spoken for 30 years in the fi elds of eco- nomics, history, business, philosophy and psychology .
nguages speaks to corporate and public audiences on the subjects of Person- al and Professional Development . talks and seminars on Leadership, Selling, Self-Esteem, Goals, Strategy, Creativity and Success Psychology bring about immediate changes .
Brian is the president of t

In [41]:
# Test the ask_question method
question1 = "What are the main ideas presented in the book?"
answer1 = agent.ask_question(question1)
print(f"\nQuestion: {question1}")
print(answer1)

question2 = "How to set goals?"
answer2 = agent.ask_question(question2)
print(f"\nQuestion: {question2}")
print(answer2)

Searching for answer to: 'What are the main ideas presented in the book?'
Most relevant section: 'Introduction/Beginning'

Question: What are the main ideas presented in the book?
Answer: The more reasons you have for 
achieving your goal (Source: 'Introduction/Beginning', Score: 0.03)
Searching for answer to: 'How to set goals?'
Most relevant section: 'Introduction/Beginning'

Question: How to set goals?
Answer: act as if it were impossible to fail (Source: 'Introduction/Beginning', Score: 0.01)


## Summary:

### Data Analysis Key Findings

*   The initial error "setting an array element with a sequence" occurred during TF-IDF vectorization when processing the provided PDF files.
*   This error was resolved by replacing the TF-IDF vectorization approach with Sentence Transformer embeddings for creating text representations.
*   The updated `BookQA` class successfully initialized a Sentence Transformer model (`all-MiniLM-L6-v2`) and used it to generate embeddings for the book sections.
*   A FAISS index was created using the generated embeddings, enabling efficient semantic search.
*   The agent successfully processed the previously problematic PDF file using the new embedding approach.
*   Core functionalities, including listing sections, asking questions, and summarizing sections, worked after the update.
*   The summarization pipeline produced warnings regarding `max_new_tokens` and `max_length` parameters but completed the task.

### Insights or Next Steps

*   The switch from TF-IDF to Sentence Transformer embeddings effectively addressed the array dimension error, suggesting that the issue was related to how TF-IDF handled sequences within the text data extracted from the PDFs.
*   Future work could involve addressing the summarization warnings for potentially better control over summary length and exploring more sophisticated text splitting techniques or larger language models for improved QA and summarization quality.


# Task
Explain and fix the "setting an array element with a sequence" error occurring during TF-IDF vectorization when processing PDF files "/content/drive/MyDrive/627260606-chatbotdoc (1).pdf" and "/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf". Additionally, modify the provided Python code to include functionality for creating flashcards from the processed book content to aid readers.

## Update `bookqa` class with flashcard functionality

### Subtask:
Add new methods to the `BookQA` class to support flashcard creation.


**Reasoning**:
Define the new methods `create_flashcards`, `_extract_flashcard_content`, and `_format_flashcard` within the `BookQA` class to support flashcard creation.



In [43]:
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import nltk
import re
import faiss
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None # This will be replaced by the embedding model
        self.tfidf_matrix = None # This will be replaced by the embeddings
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None
        self.embedding_model = None # New attribute for Sentence Transformer model


        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

        # Initialize Sentence Transformer model for embeddings
        try:
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence Transformer model loaded.")
        except Exception as e:
            print(f"Error loading Sentence Transformer model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.embedding_model = None


    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index using Sentence Transformers."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return
        if not self.embedding_model:
            print("Sentence Transformer model not loaded. Cannot vectorize sections.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return


        section_texts = list(self.chapter_sections.values())
        print(f"Creating embeddings for {len(section_texts)} sections...")

        try:
            # Create embeddings using the Sentence Transformer model
            embeddings = self.embedding_model.encode(section_texts, convert_to_numpy=True)
            self.tfidf_matrix = embeddings # Store embeddings (renamed from tfidf_matrix)

            # Create FAISS index
            dimension = embeddings.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(embeddings.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = list(self.chapter_sections.keys()) # Store original titles
            else:
                print("Embeddings have no dimensions after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during embedding creation or FAISS index creation: {e}")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.embedding_model or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Create embedding for the question
            question_vec = self.embedding_model.encode([question], convert_to_numpy=True).astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error creating question embedding or searching index: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."

    def create_flashcards(self, section_identifier=None):
        """Generates flashcards from the book content, optionally from a specific section."""
        if not self.chapter_sections:
            return "No book loaded for flashcard creation."

        sections_to_process = {}
        if section_identifier:
            section_titles = list(self.chapter_sections.keys())
            section_text = None
            section_title = None

            try:
                # Try to find by number (1-based index)
                section_index = int(section_identifier) - 1
                if 0 <= section_index < len(section_titles):
                    section_title = section_titles[section_index]
                    section_text = self.chapter_sections[section_title]
            except ValueError:
                # If not a number, try to find by partial title match
                for title in section_titles:
                    if section_identifier.lower() in title.lower():
                        section_title = title
                        section_text = self.chapter_sections[title]
                        break

            if section_text is None:
                return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."
            sections_to_process[section_title] = section_text
            print(f"Generating flashcards for section: '{section_title}'")
        else:
            sections_to_process = self.chapter_sections
            print("Generating flashcards for the entire book.")

        all_flashcards = []
        for title, text in sections_to_process.items():
            flashcards = self._extract_flashcard_content(text)
            formatted_flashcards = [self._format_flashcard(fc) for fc in flashcards]
            all_flashcards.extend(formatted_flashcards)

        if all_flashcards:
            print(f"Generated {len(all_flashcards)} potential flashcards.")
            return all_flashcards
        else:
            return "Could not generate any flashcards from the selected content."


    def _extract_flashcard_content(self, text):
        """Identifies and extracts potential flashcard material from text.
           (Basic implementation: looks for sentences ending with question marks or definitions)
        """
        # This is a very basic approach. More advanced techniques (like using NLP models
        # to identify key terms/definitions or question-answer pairs) would be better.
        # For demonstration, let's split by sentence and look for simple patterns.

        sentences = nltk.sent_tokenize(text)
        flashcard_candidates = []

        for sentence in sentences:
            # Simple pattern: sentence ending with a question mark
            if sentence.strip().endswith('?'):
                # Use the sentence as the front, and try to find a potential answer in the next sentence(s)
                front = sentence.strip()
                back = "Answer not found nearby (requires more advanced logic)" # Placeholder
                flashcard_candidates.append({'front': front, 'back': back})

            # Simple pattern: looking for "is a", "is the", "are" as potential definitions
            # This is highly heuristic and will likely produce many false positives/negatives
            definition_match = re.search(r'(.+)\s+(is a|is the|are)\s+(.+)\.', sentence, re.IGNORECASE)
            if definition_match:
                front = definition_match.group(1).strip() # Term
                back = definition_match.group(3).strip() # Definition
                flashcard_candidates.append({'front': front, 'back': back})

        return flashcard_candidates


    def _format_flashcard(self, flashcard_data):
        """Formats the extracted content into a flashcard structure."""
        # In this basic implementation, the data is already in the desired format.
        # This method serves as a placeholder for more complex formatting if needed.
        return flashcard_data


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Create Flashcards (Entire Book)")
        print("5. Create Flashcards (Specific Section)")
        print("6. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                flashcards = self.create_flashcards()
                if isinstance(flashcards, list):
                    print("\nGenerated Flashcards:")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '5':
                section_input = input("Enter the chapter/section number or name to create flashcards from: ")
                flashcards = self.create_flashcards(section_input)
                if isinstance(flashcards, list):
                    print(f"\nGenerated Flashcards for '{section_input}':")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '6':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

## Implement flashcard content extraction

### Subtask:
Develop logic within the new methods to extract key terms, concepts, or question-answer pairs from the book text.


**Reasoning**:
Refine the `_extract_flashcard_content` method to use more advanced NLP techniques and improve the extraction of flashcard front and back content.



In [44]:
import nltk
import re
from sentence_transformers import SentenceTransformer
import fitz
import faiss
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')
try:
    nltk.data.find('chunkers/maxent_ne_chunker')
except LookupError:
    nltk.download('maxent_ne_chunker')
try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')


class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None # This will be replaced by the embedding model
        self.tfidf_matrix = None # This will be replaced by the embeddings
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None
        self.embedding_model = None # New attribute for Sentence Transformer model


        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

        # Initialize Sentence Transformer model for embeddings
        try:
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence Transformer model loaded.")
        except Exception as e:
            print(f"Error loading Sentence Transformer model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.embedding_model = None


    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index using Sentence Transformers."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return
        if not self.embedding_model:
            print("Sentence Transformer model not loaded. Cannot vectorize sections.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return


        section_texts = list(self.chapter_sections.values())
        print(f"Creating embeddings for {len(section_texts)} sections...")

        try:
            # Create embeddings using the Sentence Transformer model
            embeddings = self.embedding_model.encode(section_texts, convert_to_numpy=True)
            self.tfidf_matrix = embeddings # Store embeddings (renamed from tfidf_matrix)

            # Create FAISS index
            dimension = embeddings.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(embeddings.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = list(self.chapter_sections.keys()) # Store original titles
            else:
                print("Embeddings have no dimensions after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during embedding creation or FAISS index creation: {e}")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.embedding_model or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Create embedding for the question
            question_vec = self.embedding_model.encode([question], convert_to_numpy=True).astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error creating question embedding or searching index: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."

    def create_flashcards(self, section_identifier=None):
        """Generates flashcards from the book content, optionally from a specific section."""
        if not self.chapter_sections:
            return "No book loaded for flashcard creation."

        sections_to_process = {}
        if section_identifier:
            section_titles = list(self.chapter_sections.keys())
            section_text = None
            section_title = None

            try:
                # Try to find by number (1-based index)
                section_index = int(section_identifier) - 1
                if 0 <= section_index < len(section_titles):
                    section_title = section_titles[section_index]
                    section_text = self.chapter_sections[section_title]
            except ValueError:
                # If not a number, try to find by partial title match
                for title in section_titles:
                    if section_identifier.lower() in title.lower():
                        section_title = title
                        section_text = self.chapter_sections[title]
                        break

            if section_text is None:
                return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."
            sections_to_process[section_title] = section_text
            print(f"Generating flashcards for section: '{section_title}'")
        else:
            sections_to_process = self.chapter_sections
            print("Generating flashcards for the entire book.")

        all_flashcards = []
        for title, text in sections_to_process.items():
            flashcards = self._extract_flashcard_content(text)
            formatted_flashcards = [self._format_flashcard(fc) for fc in flashcards]
            all_flashcards.extend(formatted_flashcards)

        if all_flashcards:
            print(f"Generated {len(all_flashcards)} potential flashcards.")
            return all_flashcards
        else:
            return "Could not generate any flashcards from the selected content."


    def _extract_flashcard_content(self, text):
        """Identifies and extracts potential flashcard material from text using NLP."""
        sentences = nltk.sent_tokenize(text)
        flashcard_candidates = []

        for sentence in sentences:
            # Skip very short sentences
            if len(sentence.split()) < 5:
                continue

            # Attempt to extract Question-Answer pairs using the QA pipeline
            if self.qa_pipeline:
                try:
                    # Use the sentence as the context and try to generate a question
                    # This is a simplified approach; a dedicated question generation model would be better.
                    # For now, let's try to find potential answers and formulate simple questions.
                    # A better approach would be to iterate through potential answer spans.
                    # Let's use a simple heuristic for now: look for noun phrases as potential answers.
                    tokens = nltk.word_tokenize(sentence)
                    pos_tags = nltk.pos_tag(tokens)
                    # Simple noun phrase chunking
                    grammar = "NP: {<DT>?<JJ>*<NN.*>+}"
                    cp = nltk.RegexpParser(grammar)
                    tree = cp.parse(pos_tags)

                    potential_answers = []
                    for subtree in tree.subtrees():
                        if subtree.label() == 'NP':
                            potential_answers.append(" ".join([word for (word, tag) in subtree.leaves()]))

                    # For each potential answer, try to formulate a question.
                    # This is a placeholder and needs a proper question generation approach.
                    for answer in potential_answers:
                        # Simple question generation heuristic (very basic)
                        # If the sentence contains "is a", try "What is a [term]?"
                        if " is a " in sentence.lower() and answer.lower() in sentence.lower():
                            term = sentence.lower().split(" is a ")[0].strip()
                            if len(term.split()) < 10: # Avoid very long terms
                                flashcard_candidates.append({'front': f"What is {term}?", 'back': answer})
                                continue # Move to the next potential answer

                        # If the sentence contains "are", try "What are [terms]?"
                        if " are " in sentence.lower() and answer.lower() in sentence.lower():
                             term = sentence.lower().split(" are ")[0].strip()
                             if len(term.split()) < 10: # Avoid very long terms
                                flashcard_candidates.append({'front': f"What are {term}?", 'back': answer})
                                continue # Move to the next potential answer

                        # Try a general question using the answer
                        # This is highly dependent on the sentence structure
                        # A more robust method would involve dependency parsing or a QG model
                        if len(answer.split()) > 1 and len(answer.split()) < 10: # Avoid very short/long answers
                             # Simple: "What about [answer]?" or "Tell me about [answer]?"
                             flashcard_candidates.append({'front': f"Explain: {answer}", 'back': sentence.strip()})


                except Exception as e:
                    print(f"Error during flashcard extraction (NLP): {e}")


            # Basic pattern: sentence ending with a question mark (still include as a fallback)
            if sentence.strip().endswith('?'):
                 front = sentence.strip()
                 # Try to find a short answer in the following sentence(s)
                 answer_found = False
                 for i, next_sentence in enumerate(sentences[sentences.index(sentence)+1:]):
                     if len(next_sentence.split()) < 20: # Consider relatively short sentences as potential answers
                         back = next_sentence.strip()
                         flashcard_candidates.append({'front': front, 'back': back})
                         answer_found = True
                         break
                     if i > 2: # Don't look too far ahead
                         break
                 if not answer_found:
                      flashcard_candidates.append({'front': front, 'back': "Answer not found nearby (requires more advanced logic)"}) # Placeholder


        return flashcard_candidates


    def _format_flashcard(self, flashcard_data):
        """Formats the extracted content into a flashcard structure."""
        # In this basic implementation, the data is already in the desired format.
        # This method serves as a placeholder for more complex formatting if needed.
        return flashcard_data


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Create Flashcards (Entire Book)")
        print("5. Create Flashcards (Specific Section)")
        print("6. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                flashcards = self.create_flashcards()
                if isinstance(flashcards, list):
                    print("\nGenerated Flashcards:")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '5':
                section_input = input("Enter the chapter/section number or name to create flashcards from: ")
                flashcards = self.create_flashcards(section_input)
                if isinstance(flashcards, list):
                    print(f"\nGenerated Flashcards for '{section_input}':")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '6':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


**Reasoning**:
The `_extract_flashcard_content` method is updated to use more sophisticated NLP techniques for extracting flashcard content. The next step is to test this updated method by creating flashcards from a loaded book to verify if the extraction logic works as expected.



In [45]:
# Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# Upload a book to test flashcard creation
# Use the same book path as before, assuming it loaded successfully in previous steps
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf'
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# If upload is successful, create flashcards from the first section
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to create flashcards from section: {first_section_key}")
    flashcards = agent.create_flashcards(first_section_key)

    if isinstance(flashcards, list):
        print("\nGenerated Flashcards:")
        for i, fc in enumerate(flashcards):
            print(f"Flashcard {i+1}:")
            print(f"  Front: {fc['front']}")
            print(f"  Back: {fc['back']}")
            print("-" * 20)
    else:
        print(flashcards) # Print error message
else:
    print("\nBook processing failed. Cannot create flashcards.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.
Sentence Transformer model loaded.
Models initialized.

Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Creating embeddings for 1 sections...
FAISS index created.

Attempting to create flashcards from section: Introduction/Beginning
Generating flashcards for section: 'Introduction/Beginning'


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


**Reasoning**:
The traceback indicates a `LookupError` because the `punkt_tab` resource is not found. Although `punkt` was downloaded previously, `punkt_tab` seems to be a separate requirement for sentence tokenization in this context. The next step is to download the `punkt_tab` resource using `nltk.download()`.



In [46]:
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


**Reasoning**:
The `punkt_tab` resource has been downloaded. Now, re-initialize the BookQA agent and attempt to create flashcards again to see if the `LookupError` is resolved and the flashcard extraction proceeds.



In [47]:
# Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# Upload a book to test flashcard creation
# Use the same book path as before, assuming it loaded successfully in previous steps
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf'
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# If upload is successful, create flashcards from the first section
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to create flashcards from section: {first_section_key}")
    flashcards = agent.create_flashcards(first_section_key)

    if isinstance(flashcards, list):
        print("\nGenerated Flashcards:")
        for i, fc in enumerate(flashcards):
            print(f"Flashcard {i+1}:")
            print(f"  Front: {fc['front']}")
            print(f"  Back: {fc['back']}")
            print("-" * 20)
    else:
        print(flashcards) # Print error message
else:
    print("\nBook processing failed. Cannot create flashcards.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Streaming output truncated to the last 5000 lines.

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

Error during flashcard extraction (NLP): 
**********************************************************************
  Resource averaged_perceptron_tagger_eng not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load taggers/averaged_perceptron_tagger_eng/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    -

**Reasoning**:
The traceback shows repeated `LookupError` for the resource `averaged_perceptron_tagger_eng`. Although `averaged_perceptron_tagger` was downloaded, the error specifically requests `averaged_perceptron_tagger_eng`. It seems this specific English version is needed. The next step is to download this resource.



In [48]:
try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
except LookupError:
    nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


**Reasoning**:
The required NLTK resources (`punkt`, `averaged_perceptron_tagger`, `maxent_ne_chunker`, `words`, `punkt_tab`, and `averaged_perceptron_tagger_eng`) have been downloaded. Now, re-initialize the BookQA agent with the updated flashcard extraction logic and attempt to create flashcards again to see if the NLP processes run without errors and generate flashcards.



In [49]:
# Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# Upload a book to test flashcard creation
# Use the same book path as before, assuming it loaded successfully in previous steps
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf'
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# If upload is successful, create flashcards from the first section
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to create flashcards from section: {first_section_key}")
    flashcards = agent.create_flashcards(first_section_key)

    if isinstance(flashcards, list):
        print("\nGenerated Flashcards:")
        for i, fc in enumerate(flashcards):
            print(f"Flashcard {i+1}:")
            print(f"  Front: {fc['front']}")
            print(f"  Back: {fc['back']}")
            print("-" * 20)
    else:
        print(flashcards) # Print error message
else:
    print("\nBook processing failed. Cannot create flashcards.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
L  
P
L
A
N
N
E
R
© Brian Tracy.
--------------------
Flashcard 1092:
  Front: Explain: The contents
  Back: The contents, or parts thereof, may not be reproduced in any form for any purpose without the written permission of Brian Tracy.
--------------------
Flashcard 1093:
  Front: Explain: parts thereof
  Back: The contents, or parts thereof, may not be reproduced in any form for any purpose without the written permission of Brian Tracy.
--------------------
Flashcard 1094:
  Front: Explain: any form
  Back: The contents, or parts thereof, may not be reproduced in any form for any purpose without the written permission of Brian Tracy.
--------------------
Flashcard 1095:
  Front: Explain: any purpose
  Back: The contents, or parts thereof, may not be reproduced in any form for any purpose without the written permission of Brian Tracy.
--------------------
Flashcard 1096:
  Front: Explain: Brian Tracy
  Back: The contents, or parts th

**Reasoning**:
The flashcard extraction is now working, but the generated flashcards are not ideal. The "front" is often just a phrase, and the "back" is the entire section or a large chunk of text, which is not useful for flashcards. The current subtask is to refine the extraction logic to get better key terms/questions and corresponding definitions/answers. The next step is to modify the `_extract_flashcard_content` method further to attempt to extract more meaningful pairs. This will involve trying different NLP approaches or heuristics within the existing structure.



In [50]:
import nltk
import re
from sentence_transformers import SentenceTransformer
import fitz
import faiss
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Ensure all necessary NLTK data is downloaded
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')
try:
    nltk.data.find('chunkers/maxent_ne_chunker')
except LookupError:
    nltk.download('maxent_ne_chunker')
try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
except LookupError:
    nltk.download('averaged_perceptron_tagger_eng')


class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None # This will be replaced by the embedding model
        self.tfidf_matrix = None # This will be replaced by the embeddings
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None
        self.embedding_model = None # New attribute for Sentence Transformer model


        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

        # Initialize Sentence Transformer model for embeddings
        try:
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence Transformer model loaded.")
        except Exception as e:
            print(f"Error loading Sentence Transformer model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.embedding_model = None


    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index using Sentence Transformers."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return
        if not self.embedding_model:
            print("Sentence Transformer model not loaded. Cannot vectorize sections.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return


        section_texts = list(self.chapter_sections.values())
        print(f"Creating embeddings for {len(section_texts)} sections...")

        try:
            # Create embeddings using the Sentence Transformer model
            embeddings = self.embedding_model.encode(section_texts, convert_to_numpy=True)
            self.tfidf_matrix = embeddings # Store embeddings (renamed from tfidf_matrix)

            # Create FAISS index
            dimension = embeddings.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(embeddings.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = list(self.chapter_sections.keys()) # Store original titles
            else:
                print("Embeddings have no dimensions after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during embedding creation or FAISS index creation: {e}")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.embedding_model or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Create embedding for the question
            question_vec = self.embedding_model.encode([question], convert_to_numpy=True).astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error creating question embedding or searching index: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."

    def create_flashcards(self, section_identifier=None):
        """Generates flashcards from the book content, optionally from a specific section."""
        if not self.chapter_sections:
            return "No book loaded for flashcard creation."

        sections_to_process = {}
        if section_identifier:
            section_titles = list(self.chapter_sections.keys())
            section_text = None
            section_title = None

            try:
                # Try to find by number (1-based index)
                section_index = int(section_identifier) - 1
                if 0 <= section_index < len(section_titles):
                    section_title = section_titles[section_index]
                    section_text = self.chapter_sections[section_title]
            except ValueError:
                # If not a number, try to find by partial title match
                for title in section_titles:
                    if section_identifier.lower() in title.lower():
                        section_title = title
                        section_text = self.chapter_sections[title]
                        break

            if section_text is None:
                return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."
            sections_to_process[section_title] = section_text
            print(f"Generating flashcards for section: '{section_title}'")
        else:
            sections_to_process = self.chapter_sections
            print("Generating flashcards for the entire book.")

        all_flashcards = []
        for title, text in sections_to_process.items():
            flashcards = self._extract_flashcard_content(text)
            formatted_flashcards = [self._format_flashcard(fc) for fc in flashcards]
            all_flashcards.extend(formatted_flashcards)

        if all_flashcards:
            print(f"Generated {len(all_flashcards)} potential flashcards.")
            return all_flashcards
        else:
            return "Could not generate any flashcards from the selected content."


    def _extract_flashcard_content(self, text):
        """Identifies and extracts potential flashcard material from text using NLP."""
        sentences = nltk.sent_tokenize(text)
        flashcard_candidates = []

        for sentence in sentences:
            # Skip very short sentences
            if len(sentence.split()) < 8: # Increased minimum length
                continue

            # Try to extract Term-Definition pairs using POS tagging and dependency parsing heuristics
            tokens = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(tokens)

            # Look for patterns like "Term is a Definition" or "Term refers to Definition"
            # This is still heuristic but more targeted
            definition_patterns = [
                (r'(.+)\s+(is a|is the|refers to)\s+(.+)\.', 'What is {term}?'),
                (r'(.+)\s+(are|include)\s+(.+)\.', 'What are {term}?'),
            ]

            for pattern, question_template in definition_patterns:
                match = re.search(pattern, sentence, re.IGNORECASE)
                if match:
                    term = match.group(1).strip()
                    definition = match.group(3).strip()
                    # Basic filtering for reasonable term/definition length
                    if 3 <= len(term.split()) <= 10 and len(definition.split()) >= 5:
                         flashcard_candidates.append({'front': question_template.format(term=term), 'back': definition})
                         # Once a definition is found, move to the next sentence
                         break # Break out of pattern loop, continue to next sentence

            # As a fallback, consider sentences with question marks
            if sentence.strip().endswith('?'):
                front = sentence.strip()
                # Try to find a concise answer in the following sentence(s)
                answer_found = False
                for i, next_sentence in enumerate(sentences[sentences.index(sentence)+1:]):
                    # Consider sentences that are not too long as potential answers
                    if 5 <= len(next_sentence.split()) <= 30:
                        back = next_sentence.strip()
                        flashcard_candidates.append({'front': front, 'back': back})
                        answer_found = True
                        break
                    if i > 3: # Don't look too far ahead
                        break
                if not answer_found:
                     # If no good answer found, use a placeholder or the original sentence (truncated)
                     back = sentence.strip()[:100] + "..." if len(sentence) > 100 else sentence.strip()
                     flashcard_candidates.append({'front': front, 'back': f"Answer not found nearby. Relevant context: {back}"})


        # Additional heuristic: Extracting key terms and their surrounding sentence as definition
        # Identify potential key terms (e.g., prominent noun phrases)
        terms_to_consider = set()
        tokens = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(tokens)
        grammar = "NP: {<DT>?<JJ>*<NNP>+|<DT>?<JJ>*<NN>+<NNS>?}" # More specific NP pattern
        cp = nltk.RegexpParser(grammar)
        tree = cp.parse(pos_tags)

        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                term = " ".join([word for (word, tag) in subtree.leaves()])
                # Filter terms based on length and common words
                if 2 <= len(term.split()) <= 5 and term.lower() not in ['the', 'a', 'is', 'are', 'this', 'that', 'it']:
                     terms_to_consider.add(term)

        # Create flashcards for key terms by finding sentences where they appear
        for term in terms_to_consider:
             for sentence in sentences:
                  if re.search(r'\b' + re.escape(term) + r'\b', sentence, re.IGNORECASE):
                       # Use the term as the front, and the sentence (or a part of it) as the back
                       front = term.strip()
                       back = sentence.strip()
                       # Avoid creating duplicate flashcards if already captured by other patterns
                       if {'front': front, 'back': back} not in flashcard_candidates:
                           flashcard_candidates.append({'front': front, 'back': back})
                           break # Move to the next term after finding one sentence


        return flashcard_candidates


    def _format_flashcard(self, flashcard_data):
        """Formats the extracted content into a flashcard structure."""
        # In this basic implementation, the data is already in the desired format.
        # This method serves as a placeholder for more complex formatting if needed.
        return flashcard_data


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            return "No book loaded for study/review."

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Create Flashcards (Entire Book)")
        print("5. Create Flashcards (Specific Section)")
        print("6. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                flashcards = self.create_flashcards()
                if isinstance(flashcards, list):
                    print("\nGenerated Flashcards:")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '5':
                section_input = input("Enter the chapter/section number or name to create flashcards from: ")
                flashcards = self.create_flashcards(section_input)
                if isinstance(flashcards, list):
                    print(f"\nGenerated Flashcards for '{section_input}':")
                    for i, fc in enumerate(flashcards):
                        print(f"Flashcard {i+1}:")
                        print(f"  Front: {fc['front']}")
                        print(f"  Back: {fc['back']}")
                        print("-" * 20)
                else:
                    print(flashcards) # Print error message
            elif choice == '6':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

**Reasoning**:
The `_extract_flashcard_content` method has been further refined with more targeted heuristics for extracting flashcard content. The next logical step in completing the subtask is to test this updated method by generating flashcards from the loaded book again. This will help evaluate the effectiveness of the new extraction logic and identify if further adjustments are needed.



In [51]:
# Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# Upload a book to test flashcard creation
# Use the same book path as before, assuming it loaded successfully in previous steps
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf'
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# If upload is successful, create flashcards from the first section
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to create flashcards from section: {first_section_key}")
    flashcards = agent.create_flashcards(first_section_key)

    if isinstance(flashcards, list):
        print("\nGenerated Flashcards:")
        for i, fc in enumerate(flashcards):
            print(f"Flashcard {i+1}:")
            print(f"  Front: {fc['front']}")
            print(f"  Back: {fc['back']}")
            print("-" * 20)
    else:
        print(flashcards) # Print error message
else:
    print("\nBook processing failed. Cannot create flashcards.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.
Sentence Transformer model loaded.
Models initialized.

Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Creating embeddings for 1 sections...
FAISS index created.

Attempting to create flashcards from section: Introduction/Beginning
Generating flashcards for section: 'Introduction/Beginning'
Generated 280 potential flashcards.

Generated Flashcards:
Flashcard 1:
  Front: In each case the question you ask yourself is: “What do I really want?
  Back: What are my 
real goals in life?”
You are asked to think about your major goals based on ten goal-achieving criteria.
--------------------
Flashcard 2:
  Front: How would you spend your time, where would you go, what would you do, if 
you found out today that you only had six months to live?
  Ba

## Format flashcards

### Subtask:
Create a method to format the extracted content into a flashcard format (e.g., a list of tuples or a dictionary).


**Reasoning**:
Review the output format of `_extract_flashcard_content` and the `_format_flashcard` method to ensure the output is a list of dictionaries with 'front' and 'back' keys and the format method simply returns the input.



In [52]:
# Reviewing the methods in the existing class definition
# The _extract_flashcard_content method returns a list of dictionaries with 'front' and 'back' keys.
# The _format_flashcard method currently returns the input dictionary directly, which aligns with the desired output format.
# No code modification is needed for this subtask.

## Add a user interface for flashcards

### Subtask:
Integrate the flashcard functionality into the `study_review` method or create a new method for users to access and view the generated flashcards.


**Reasoning**:
Integrate the flashcard functionality into the `study_review` method by adding new menu options for creating and viewing flashcards and including logic to display the generated flashcards.



In [53]:
import nltk
import re
from sentence_transformers import SentenceTransformer
import fitz
import faiss
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Ensure all necessary NLTK data is downloaded
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')
try:
    nltk.data.find('chunkers/maxent_ne_chunker')
except LookupError:
    nltk.download('maxent_ne_chunker')
try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')
try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
except LookupError:
    nltk.download('averaged_perceptron_tagger_eng')


class BookQA:
    def __init__(self):
        self.book_text = ""
        self.chapter_sections = {} # To store chapter/section titles and their text
        self.vectorizer = None # This will be replaced by the embedding model
        self.tfidf_matrix = None # This will be replaced by the embeddings
        self.index = None # FAISS index for vector search
        self.qa_pipeline = None
        self.summarizer_pipeline = None
        self.tokenizer = None
        self.summarizer_model = None
        self.embedding_model = None # New attribute for Sentence Transformer model
        self.generated_flashcards = [] # To store generated flashcards


        # Initialize NLP pipelines and models
        print("Initializing NLP models...")
        self.initialize_models()
        print("Models initialized.")

    def initialize_models(self):
        # Initialize Question Answering pipeline
        # Using a smaller model for demonstration; consider a larger one for better results
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            print("Question Answering model loaded.")
        except Exception as e:
            print(f"Error loading QA model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.qa_pipeline = None

        # Initialize Summarization model and tokenizer
        # Using a smaller T5 model; consider t5-large or bart-large-cnn for better summaries
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("t5-small")
            self.summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
            self.summarizer_pipeline = pipeline("summarization", model=self.summarizer_model, tokenizer=self.tokenizer)
            print("Summarization model loaded.")
        except Exception as e:
            print(f"Error loading Summarization model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.tokenizer = None
            self.summarizer_model = None
            self.summarizer_pipeline = None

        # Initialize Sentence Transformer model for embeddings
        try:
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence Transformer model loaded.")
        except Exception as e:
            print(f"Error loading Sentence Transformer model: {e}")
            print("Please ensure you have sufficient resources and internet connection.")
            self.embedding_model = None


    def clean_text(self, text):
        """Basic text cleaning."""
        text = text.lower() # Convert to lowercase
        text = re.sub(r'[^a-z0-9\s]', '', text) # Remove non-alphanumeric characters except spaces
        text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
        return text

    def upload_book(self, file_path):
        """Reads content from a PDF file."""
        try:
            doc = fitz.open(file_path)
            self.book_text = ""
            print(f"Reading {doc.page_count} pages from {file_path}...")
            for page_num in range(doc.page_count):
                page = doc.load_page(page_num)
                self.book_text += page.get_text()
            print("Finished reading book.")
            self.process_book_text()
        except Exception as e:
            print(f"Error reading file: {e}")
            self.book_text = ""
            self.chapter_sections = {}

    def process_book_text(self):
        """Splits book text into chapters/sections and prepares for search."""
        if not self.book_text:
            print("No book text loaded to process.")
            return

        print("Processing book text...")
        # Simple approach to split into sections based on common patterns (e.g., "Chapter X", "Section Y")
        # This is a basic implementation; a more robust parser might be needed for complex books.
        sections = re.split(r'(Chapter \d+|Section \d+)', self.book_text, flags=re.IGNORECASE)

        current_title = "Introduction/Beginning"
        current_text = ""
        self.chapter_sections = {} # Clear existing sections

        for i, part in enumerate(sections):
            if i % 2 == 1: # This part is likely a title
                if current_text.strip():
                    self.chapter_sections[current_title] = current_text.strip()
                current_title = part.strip()
                current_text = ""
            else: # This part is the content
                current_text += part

        if current_text.strip(): # Add the last section
             self.chapter_sections[current_title] = current_text.strip()

        # Handle cases where no clear chapters/sections are found (e.g., a simple document)
        if not self.chapter_sections and self.book_text:
             # Split into chunks for search
             chunk_size = 2000 # characters
             chunks = [self.book_text[i:i + chunk_size] for i in range(0, len(self.book_text), chunk_size)]
             self.chapter_sections = {f"Chunk {i+1}": chunk for i, chunk in enumerate(chunks)}
             print(f"Book text split into {len(chunks)} chunks.")

        print(f"Identified {len(self.chapter_sections)} chapters/sections or chunks.")

        # Prepare for vector search
        self.prepare_for_search()

    def prepare_for_search(self):
        """Vectorizes the text and creates a FAISS index using Sentence Transformers."""
        if not self.chapter_sections:
            print("No sections to vectorize.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return
        if not self.embedding_model:
            print("Sentence Transformer model not loaded. Cannot vectorize sections.")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            return


        section_texts = list(self.chapter_sections.values())
        print(f"Creating embeddings for {len(section_texts)} sections...")

        try:
            # Create embeddings using the Sentence Transformer model
            embeddings = self.embedding_model.encode(section_texts, convert_to_numpy=True)
            self.tfidf_matrix = embeddings # Store embeddings (renamed from tfidf_matrix)

            # Create FAISS index
            dimension = embeddings.shape[1]
            if dimension > 0:
                self.index = faiss.IndexFlatL2(dimension) # Using L2 distance (Euclidean)
                self.index.add(embeddings.astype('float32')) # FAISS requires float32
                print("FAISS index created.")
                # Store the titles corresponding to the vectorized texts for later lookup
                self._vectorized_section_titles = list(self.chapter_sections.keys()) # Store original titles
            else:
                print("Embeddings have no dimensions after vectorization. Cannot create FAISS index.")
                self.index = None
                self._vectorized_section_titles = []

        except Exception as e:
            print(f"Error during embedding creation or FAISS index creation: {e}")
            self.tfidf_matrix = None # Renamed to embeddings
            self.index = None
            self._vectorized_section_titles = []


    def ask_question(self, question):
        """Answers a question based on the book content."""
        if not self.book_text:
            return "Please upload a book first."
        if not self.qa_pipeline:
             return "Question Answering model is not loaded. Please check initialization."
        if not self.embedding_model or not self.index or not hasattr(self, '_vectorized_section_titles') or not self._vectorized_section_titles:
             return "Book content not vectorized or index not created. Cannot answer questions."


        print(f"Searching for answer to: '{question}'")

        # Find the most relevant section using the FAISS index
        try:
            # Create embedding for the question
            question_vec = self.embedding_model.encode([question], convert_to_numpy=True).astype('float32')
            D, I = self.index.search(question_vec, 1) # Search for the top 1 most similar section
        except Exception as e:
            print(f"Error creating question embedding or searching index: {e}")
            return "Could not process the question for searching."


        if I.size == 0 or I[0][0] == -1:
            return "Could not find a relevant section in the book."

        relevant_section_index = I[0][0]
        # Use the stored vectorized section titles to get the correct title
        relevant_section_title = self._vectorized_section_titles[relevant_section_index]
        relevant_section_text = self.chapter_sections[relevant_section_title] # Use original text for QA


        print(f"Most relevant section: '{relevant_section_title}'")

        # Use the QA pipeline on the relevant section
        try:
            # The QA pipeline has context length limitations. We might need to
            # further refine the relevant text chunk for the QA model.
            # For simplicity, using the whole section here, which might fail for very long sections.
            # A better approach would be to split the section into smaller paragraphs.
            # For this example, let's limit the context length for the pipeline
            max_context_length = self.qa_pipeline.model.config.max_position_embeddings if hasattr(self.qa_pipeline.model.config, 'max_position_embeddings') else 512
            # Let's roughly estimate tokens by characters for now (approx 4 chars per token)
            max_chars = max_context_length * 4
            context = relevant_section_text[:max_chars]


            answer = self.qa_pipeline(question=question, context=context) # Use original question for QA pipeline
            return f"Answer: {answer['answer']} (Source: '{relevant_section_title}', Score: {answer['score']:.2f})"
        except Exception as e:
            print(f"Error during QA pipeline: {e}")
            # Fallback: Return the relevant section
            return f"Could not generate a specific answer. Relevant section from '{relevant_section_title}':\n{relevant_section_text[:500]}..." # Show beginning of section


    def list_sections(self):
        """Lists the identified chapters/sections."""
        if not self.chapter_sections:
            return "No chapters or sections identified yet. Please upload a book."
        print("Chapters/Sections:")
        for i, title in enumerate(self.chapter_sections.keys()):
            print(f"{i+1}. {title}")

    def summarize_section(self, section_identifier):
        """Summarizes a specified chapter or section."""
        if not self.chapter_sections:
            return "No chapters or sections available to summarize. Please upload a book."
        if not self.summarizer_pipeline:
            return "Summarization model is not loaded. Please check initialization."

        section_titles = list(self.chapter_sections.keys())
        section_text = None
        section_title = None

        try:
            # Try to find by number (1-based index)
            section_index = int(section_identifier) - 1
            if 0 <= section_index < len(section_titles):
                section_title = section_titles[section_index]
                section_text = self.chapter_sections[section_title]
        except ValueError:
            # If not a number, try to find by partial title match
            for title in section_titles:
                if section_identifier.lower() in title.lower():
                    section_title = title
                    section_text = self.chapter_sections[title]
                    break

        if section_text is None:
            return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."

        print(f"Summarizing section: '{section_title}'")

        # Summarization models have input length limits. We need to handle long sections.
        # A common approach is to split the text into smaller chunks, summarize each chunk,
        # and then optionally summarize the summaries.
        # For simplicity here, we'll just truncate or split the text for the summarizer input.

        # Let's split the text into smaller chunks and summarize each chunk
        chunk_size = 1000 # characters per chunk for summarization input
        chunks = [section_text[i:i + chunk_size] for i in range(0, len(section_text), chunk_size)]

        all_summaries = []
        print(f"Splitting section into {len(chunks)} chunks for summarization...")
        for i, chunk in enumerate(chunks):
             # Need to handle potential tokenizer limits more robustly
             # A simple approach: if a chunk is too long after tokenization, summarize a smaller part
             input_tokens = self.tokenizer(chunk, return_tensors="pt", max_length=self.tokenizer.model_max_length, truncation=True).input_ids
             if input_tokens.shape[1] > self.tokenizer.model_max_length:
                 print(f"Warning: Chunk {i+1} is too long, truncating for summarization.")
                 # Further split or refine truncation if necessary
                 # For this example, just use the truncated version by the tokenizer

             try:
                summary = self.summarizer_pipeline(chunk, max_length=150, min_length=30, do_sample=False)
                all_summaries.append(summary[0]['summary_text'])
                print(f"Summarized chunk {i+1}/{len(chunks)}")
             except Exception as e:
                 print(f"Error summarizing chunk {i+1}: {e}")
                 all_summaries.append(f"[Error summarizing this part: {str(e)[:100]}...]") # Add error indicator

        # Combine the summaries
        if all_summaries:
            combined_summary = "\n".join(all_summaries)
            return f"Summary of '{section_title}':\n{combined_summary}"
        else:
            return f"Could not generate a summary for '{section_title}'. No valid chunks were processed."

    def create_flashcards(self, section_identifier=None):
        """Generates flashcards from the book content, optionally from a specific section."""
        if not self.chapter_sections:
            return "No book loaded for flashcard creation."

        sections_to_process = {}
        if section_identifier:
            section_titles = list(self.chapter_sections.keys())
            section_text = None
            section_title = None

            try:
                # Try to find by number (1-based index)
                section_index = int(section_identifier) - 1
                if 0 <= section_index < len(section_titles):
                    section_title = section_titles[section_index]
                    section_text = self.chapter_sections[section_title]
            except ValueError:
                # If not a number, try to find by partial title match
                for title in section_titles:
                    if section_identifier.lower() in title.lower():
                        section_title = title
                        section_text = self.chapter_sections[title]
                        break

            if section_text is None:
                return f"Could not find a section matching '{section_identifier}'. Use `list_sections()` to see available sections."
            sections_to_process[section_title] = section_text
            print(f"Generating flashcards for section: '{section_title}'")
        else:
            sections_to_process = self.chapter_sections
            print("Generating flashcards for the entire book.")

        all_flashcards = []
        for title, text in sections_to_process.items():
            flashcards = self._extract_flashcard_content(text)
            formatted_flashcards = [self._format_flashcard(fc) for fc in flashcards]
            all_flashcards.extend(formatted_flashcards)

        if all_flashcards:
            print(f"Generated {len(all_flashcards)} potential flashcards.")
            self.generated_flashcards = all_flashcards # Store generated flashcards
            return all_flashcards
        else:
            self.generated_flashcards = []
            return "Could not generate any flashcards from the selected content."


    def _extract_flashcard_content(self, text):
        """Identifies and extracts potential flashcard material from text using NLP."""
        sentences = nltk.sent_tokenize(text)
        flashcard_candidates = []

        for sentence in sentences:
            # Skip very short sentences
            if len(sentence.split()) < 8: # Increased minimum length
                continue

            # Try to extract Term-Definition pairs using POS tagging and dependency parsing heuristics
            tokens = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(tokens)

            # Look for patterns like "Term is a Definition" or "Term refers to Definition"
            # This is still heuristic but more targeted
            definition_patterns = [
                (r'(.+)\s+(is a|is the|refers to)\s+(.+)\.', 'What is {term}?'),
                (r'(.+)\s+(are|include)\s+(.+)\.', 'What are {term}?'),
            ]

            for pattern, question_template in definition_patterns:
                match = re.search(pattern, sentence, re.IGNORECASE)
                if match:
                    term = match.group(1).strip()
                    definition = match.group(3).strip()
                    # Basic filtering for reasonable term/definition length
                    if 3 <= len(term.split()) <= 10 and len(definition.split()) >= 5:
                         flashcard_candidates.append({'front': question_template.format(term=term), 'back': definition})
                         # Once a definition is found, move to the next sentence
                         break # Break out of pattern loop, continue to next sentence

            # As a fallback, consider sentences with question marks
            if sentence.strip().endswith('?'):
                front = sentence.strip()
                # Try to find a concise answer in the following sentence(s)
                answer_found = False
                for i, next_sentence in enumerate(sentences[sentences.index(sentence)+1:]):
                    # Consider sentences that are not too long as potential answers
                    if 5 <= len(next_sentence.split()) <= 30:
                        back = next_sentence.strip()
                        flashcard_candidates.append({'front': front, 'back': back})
                        answer_found = True
                        break
                    if i > 3: # Don't look too far ahead
                        break
                if not answer_found:
                     # If no good answer found, use a placeholder or the original sentence (truncated)
                     back = sentence.strip()[:100] + "..." if len(sentence) > 100 else sentence.strip()
                     flashcard_candidates.append({'front': front, 'back': f"Answer not found nearby. Relevant context: {back}"})


        # Additional heuristic: Extracting key terms and their surrounding sentence as definition
        # Identify potential key terms (e.g., prominent noun phrases)
        terms_to_consider = set()
        tokens = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(tokens)
        grammar = "NP: {<DT>?<JJ>*<NNP>+|<DT>?<JJ>*<NN>+<NNS>?}" # More specific NP pattern
        cp = nltk.RegexpParser(grammar)
        tree = cp.parse(pos_tags)

        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                term = " ".join([word for (word, tag) in subtree.leaves()])
                # Filter terms based on length and common words
                if 2 <= len(term.split()) <= 5 and term.lower() not in ['the', 'a', 'is', 'are', 'this', 'that', 'it']:
                     terms_to_consider.add(term)

        # Create flashcards for key terms by finding sentences where they appear
        for term in terms_to_consider:
             for sentence in sentences:
                  if re.search(r'\b' + re.escape(term) + r'\b', sentence, re.IGNORECASE):
                       # Use the term as the front, and the sentence (or a part of it) as the back
                       front = term.strip()
                       back = sentence.strip()
                       # Avoid creating duplicate flashcards if already captured by other patterns
                       if {'front': front, 'back': back} not in flashcard_candidates:
                           flashcard_candidates.append({'front': front, 'back': back})
                           break # Move to the next term after finding one sentence


        return flashcard_candidates


    def _format_flashcard(self, flashcard_data):
        """Formats the extracted content into a flashcard structure."""
        # In this basic implementation, the data is already in the desired format.
        # This method serves as a placeholder for more complex formatting if needed.
        return flashcard_data

    def view_flashcards(self):
        """Displays the generated flashcards."""
        if not self.generated_flashcards:
            return "No flashcards have been generated yet. Please create them first."

        print("\n--- Generated Flashcards ---")
        for i, fc in enumerate(self.generated_flashcards):
            print(f"Flashcard {i+1}:")
            print(f"  Front: {fc['front']}")
            print(f"  Back: {fc['back']}")
            print("-" * 20)
        return f"Displayed {len(self.generated_flashcards)} flashcards."


    def study_review(self):
        """Provides options for studying and reviewing."""
        if not self.chapter_sections:
            print("No book loaded for study/review.")
            return

        print("\n--- Study & Review Options ---")
        print("1. List Chapters/Sections")
        print("2. Summarize a specific Chapter/Section")
        print("3. Ask a question about the book")
        print("4. Create Flashcards (Entire Book)")
        print("5. Create Flashcards (Specific Section)")
        print("6. View Generated Flashcards") # New option
        print("7. Exit Study/Review")

        while True:
            choice = input("Enter your choice: ")
            if choice == '1':
                self.list_sections()
            elif choice == '2':
                section_input = input("Enter the chapter/section number or name to summarize: ")
                summary = self.summarize_section(section_input)
                print(summary)
            elif choice == '3':
                question = input("What is your question about the book? ")
                answer = self.ask_question(question)
                print(answer)
            elif choice == '4':
                flashcard_result = self.create_flashcards()
                if isinstance(flashcard_result, str):
                    print(flashcard_result) # Print error message
                else:
                    print("Flashcards generated. Use option 6 to view them.")
            elif choice == '5':
                section_input = input("Enter the chapter/section number or name to create flashcards from: ")
                flashcard_result = self.create_flashcards(section_input)
                if isinstance(flashcard_result, str):
                    print(flashcard_result) # Print error message
                else:
                     print("Flashcards generated. Use option 6 to view them.")
            elif choice == '6': # New option
                view_result = self.view_flashcards()
                print(view_result)
            elif choice == '7':
                print("Exiting Study/Review.")
                break
            else:
                print("Invalid choice. Please try again.")

# --- How to use the agent in Colab ---

# 1. Mount Google Drive to access your files (optional, but common)
# from google.colab import drive
# drive.mount('/content/drive')

# 2. Create an instance of the BookQA agent
# agent = BookQA()

# 3. Upload a book (replace the path with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
# book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # <--- **CHANGE THIS TO YOUR BOOK PATH**
# print(f"\nAttempting to upload book from: {book_path}")
# agent.upload_book(book_path)

# --- Now you can interact with the agent ---

# To enter the study/review mode and test flashcards:
# if agent.book_text:
#     agent.study_review()
# else:
#     print("\nBook was not loaded successfully. Please check the file path and format.")

## Test and refine flashcard creation

### Subtask:
Test the flashcard creation with different sections of a book to ensure relevant and well-formatted flashcards are generated. Refine the extraction logic as needed.


**Reasoning**:
Create a new instance of the BookQA agent to load the updated code, upload a book, and then list the sections to prepare for testing flashcard creation.



In [54]:
# 2. Create an instance of the BookQA agent to load the updated code
agent = BookQA()

# 3. Upload a book (replace the path with your file path)
# Make sure the PDF is in your Google Drive or uploaded directly to the Colab runtime.
book_path = '/content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf' # Using the problematic file again
print(f"\nAttempting to upload book from: {book_path}")
agent.upload_book(book_path)

# 4. List sections to see available options
if agent.chapter_sections:
    print("\nBook processed successfully. Listing sections:")
    agent.list_sections()
else:
    print("\nBook processing failed.")

Initializing NLP models...


Device set to use cuda:0


Question Answering model loaded.


Device set to use cuda:0


Summarization model loaded.
Sentence Transformer model loaded.
Models initialized.

Attempting to upload book from: /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf
Reading 89 pages from /content/drive/MyDrive/229250892-Brian-Tracy-Goal-Planner.pdf...
Finished reading book.
Processing book text...
Identified 1 chapters/sections or chunks.
Creating embeddings for 1 sections...
FAISS index created.

Book processed successfully. Listing sections:
Chapters/Sections:
1. Introduction/Beginning


**Reasoning**:
Call the create_flashcards method for the identified section and then call the view_flashcards method to display them and examine their quality.



In [55]:
# 5. Call create_flashcards for the first section
if agent.chapter_sections:
    first_section_key = list(agent.chapter_sections.keys())[0]
    print(f"\nAttempting to create flashcards from section: {first_section_key}")
    flashcard_result = agent.create_flashcards(first_section_key)

    # 6. View the generated flashcards
    view_result = agent.view_flashcards()
    print(view_result)

    # Note down observations on the quality of flashcards here after viewing the output.
    # Observation: The generated flashcards are based on simple pattern matching and key term extraction.
    # They might not be perfect Q&A pairs or definitions and may require further refinement of the extraction logic.
    # For this specific book ("Brian Tracy Goal Planner"), the "Introduction/Beginning" section
    # seems to contain mostly introductory text and book structure, which might not yield
    # many high-quality flashcards with the current heuristic-based extraction.
    # More specific sections on "Goal Setting" or "Action Planning" might yield better results.
    # However, given there's only one identified section, the current test is limited.

else:
    print("\nBook processing failed. Cannot create flashcards.")


Attempting to create flashcards from section: Introduction/Beginning
Generating flashcards for section: 'Introduction/Beginning'
Generated 280 potential flashcards.

--- Generated Flashcards ---
Flashcard 1:
  Front: In each case the question you ask yourself is: “What do I really want?
  Back: What are my 
real goals in life?”
You are asked to think about your major goals based on ten goal-achieving criteria.
--------------------
Flashcard 2:
  Front: How would you spend your time, where would you go, what would you do, if 
you found out today that you only had six months to live?
  Back: Answer not found nearby. Relevant context: How would you spend your time, where would you go, what would you do, if 
you found out today that y...
--------------------
Flashcard 3:
  Front: What would you do differently if you won one million dollars cash, tax free, 
tomorrow?
  Back: What have you always wanted to do but been afraid to attempt?
--------------------
Flashcard 4:
  Front: What have y

## Summary:

### Data Analysis Key Findings

*   The original task involved fixing a "setting an array element with a sequence" error during TF-IDF vectorization. However, this specific error was not encountered or addressed in the provided solving steps, as the focus shifted entirely to adding flashcard functionality.
*   Flashcard creation functionality was successfully added to the `BookQA` class, including methods for creating, extracting content, and formatting flashcards.
*   The flashcard extraction logic uses NLTK for sentence tokenization, POS tagging, and basic noun phrase chunking, along with heuristic pattern matching for term-definition pairs and question-answer structures.
*   Initial testing of the flashcard creation revealed issues with NLTK resource availability (`punkt_tab`, `averaged_perceptron_tagger_eng`), which were resolved by downloading the necessary resources.
*   The heuristic-based flashcard extraction method, even after refinement with more specific patterns and key term extraction, produced flashcards of inconsistent quality. Many were not ideal Q\&A pairs or definitions, highlighting the limitations of this approach for complex text.
*   Testing the flashcard creation with different sections was a required step, but the specific book used in the final testing phase ("Brian Tracy Goal Planner") was processed into only a single section, preventing comprehensive testing across varied content.

### Insights or Next Steps

*   The current heuristic-based flashcard extraction is insufficient for consistently generating high-quality flashcards from arbitrary book text. More advanced NLP techniques, such as dedicated Question Generation (QG) models, Information Extraction (IE), or more sophisticated semantic analysis, would be required to significantly improve flashcard quality.
*   To fully evaluate and refine the flashcard creation logic, it is necessary to test with books that are parsed into multiple distinct sections by the `process_book_text` method. Alternatively, the sectioning logic could be improved to create smaller, more manageable chunks of text for processing.
